## 3. 데이터 수집

앗, 그런데 데이터를 어디서 구해야할까요? 여러 사이트가 있지만 오늘은 ~~[서울 열린데이터 광장](https://data.seoul.go.kr/) 데이터를 사용할 예정~~이었지만 아쉽게도 22년 4월 즈음부터 제공 데이터의 형식과 내용이 바뀌었어요. 일단 CCTV 데이터만 다운로드 하시고, 인구통계는 github의 같은 페이지에 올려 두었으니 다운로드해서 사용하시기 바랍니다.

- [서울시 자치구 연도별 CCTV 설치 현황](https://data.seoul.go.kr/dataList/OA-2734/F/1/datasetView.do)
- [서울시 주민등록인구 통계_github](https://github.com/Team-COSADAMA/Data-Science-Intro/tree/main/week2): download 또는 view raw -> 해당 페이지 우클릭 후 다른 이름으로 저장
- ~~[서울시 주민등록인구 통계](http://data.seoul.go.kr/dataList/419/S/2/datasetView.do): 12년 ~ 21년을 '조회'하고 xls 형식으로 다운로드~~ 

혹시 다른 지역으로 해보시려면 웬만한 공공데이터가 모여 있는 [공공데이터포털](https://www.data.go.kr/)에서 찾으시면 됩니다.

## 4. 데이터 가공

### 4.1. 파일 읽어오기

하나는 csv(콤마로 구분된 텍스트 파일)이고, 다른 하나는 엑셀(.xlsx or .xls)입니다. 파이썬으로 이 둘을 모두 읽어볼 수 있는 방법을 익혀볼게요. pandas는 read_csv, read_excel 같은 단순한 코드 하나만으로 다양한 파일을 읽을 수 있습니다!

In [ ]:
import pandas as pd    
import numpy as np

In [ ]:
CCTV_Seoul = pd.read_csv('/서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv', 
                        encoding='cp949')
CCTV_Seoul.head()  # 데이터 앞의 5줄만 보여주세요!

,"※ 2021.7.31. 현황을 기준으로 각 연도별 설치된 CCTV 수량. 교체(저화질교체, 성능개선)는 최초설치연도가 아닌 교체년도 수량에 포함",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
1,계,"77,032","7,667","2,200","3,491","4,439","6,582","8,129","9,947","9,876","11,961","11,132","1,608"
2,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
3,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289
4,용산구,"2,383",34,71,234,125,221,298,351,125,307,617,0


후.. 이거 만든 사람 누군지 모르겠지만 맨 처음 행row이 컬럼column이 아니군요. 우리는 구별 비교를 할 예정이니 1행의 '계'에 해당하는 데이터도 필요 없네요. 어쩔 수 없죠. 처리해줍시다!

In [ ]:
CCTV_Seoul = pd.read_csv('/서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv', 
                        skiprows = [0,2],       # ※표시 있는 행, '계 있는 행 없애줘
                        header = 0,             # 컬럼은 위에서 삭제하고 남은 0번째 줄로 해줘
                        encoding='cp949')
CCTV_Seoul.head()  # 데이터 앞의 5줄만 보여줘!

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,종로구,"1,772",813,0,0,210,150,1,261,85,9,200,43
1,중 구,"2,333",16,114,87,77,236,240,372,386,155,361,289
2,용산구,"2,383",34,71,234,125,221,298,351,125,307,617,0
3,성동구,"3,602",448,125,212,105,339,310,874,390,262,461,76
4,광진구,"2,588",35,57,100,187,98,52,675,465,712,175,32


항상 데이터들을 살펴보는 것을 잊지마세요. 데이터가 무엇을 의미하는지도 중요하니까요. 대충 강남구가 다른 곳에 비해서 많이 설치되어있네. 추이도 볼 수 있는데, 설치수가 줄어든 것도 보이고 늘어난 곳도 보이네요.


저는 보통 데이터를 import한 다음 제일 먼저 **info**를 통해 자료형, 결측치를 살펴본답니다. 급하게 데이터 타입을 맞추거나 분석한 후에 결측치가 있으면 굉장히 난감하거든요. 분석을 처음부터 다시 해야할 수 있어요.

In [ ]:
CCTV_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구분        25 non-null     object
 1   총계        25 non-null     object
 2   2012년 이전  25 non-null     object
 3   2012년     25 non-null     int64 
 4   2013년     25 non-null     int64 
 5   2014년     25 non-null     int64 
 6   2015년     25 non-null     int64 
 7   2016년     25 non-null     int64 
 8   2017년     25 non-null     int64 
 9   2018년     25 non-null     int64 
 10  2019년     25 non-null     int64 
 11  2020년     25 non-null     object
 12  2021년     25 non-null     int64 
dtypes: int64(9), object(4)
memory usage: 2.7+ KB


이런~ 총계, 2012년 이전, 2020년 데이터가 object(str)군요. 공공기관 데이터는 여러 해에 걸쳐 여러 담당자가 수집하다 보니 자료형이 어긋난 경우가 종종 있어요. (물론 배포하기 전에 신경쓰는게 맞지만요)

모든 데이터에 null값(NaN) 값도 없군요. 좋아요. 저 둘의 dtype을 int로 바꿔줍시다.

In [ ]:
CCTV_Seoul['총계'] = CCTV_Seoul['총계'].astype(int)
CCTV_Seoul['2012년 이전'] = CCTV_Seoul['2012년 이전'].astype(int)
CCTV_Seoul['2020년'] = CCTV_Seoul['2020년'].astype(int)

CCTV_Seoul.info()

ValueError: invalid literal for int() with base 10: '1,772'

!!!! 분명 잘 한 것 같은데 에러가 나오네요? 이럴 때엔 에러를 잘 살펴보면 문제를 알아낼 수 있어요. int화 하는데 '1,772'가 적절하지 않다고 나오네요. 작은 따옴표로 묶여 있으니 문자를 int로 바꿀 때 생긴 문제겠군요.


csv는 comma separated value. 그러니까 값이 comma로 구별되는 데이터예요. pandas 입장에서 생각하면 comma로 구분해 가져온 데이터프레임에 굳~이 ,를 꼭 명시해준 문자열 데이터가 들어간 상태예요. 문자열은 별개로 인식해서 괜찮지만 숫자로 바꾸면 문자로 다시 인식하거나 ,(comma)를 구분자로 잘못 인식할 수 있어요. 이럴때 우리는 크게 두가지 방법을 씁니다.

- **read_csv(thousands = ','): 천 단위에 들어간 ,는 무시하고 숫자로 인식해줘!**
- **df['해당 컬럼'].str.replace(',', ''): 문자열 ','를 공백으로 바꿔줘!**

분석 시작한지 얼마 되지 않았으니 read_csv로 다시 읽어옵시다.

In [ ]:
CCTV_Seoul = pd.read_csv('/서울시CCTV설치운영현황(자치구)_년도별_210731기준.csv', 
                        skiprows = [0,2],       # ※표시 있는 행, '계 있는 행 없애줘
                        header = 0,             # 컬럼은 위에서 삭제하고 남은 0번째 줄로 해줘
                        thousands = ',',
                        encoding='cp949')
CCTV_Seoul.head()  # 데이터 앞의 5줄만 보여줘!

,구분,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
1,중 구,2333,16,114,87,77,236,240,372,386,155,361,289
2,용산구,2383,34,71,234,125,221,298,351,125,307,617,0
3,성동구,3602,448,125,212,105,339,310,874,390,262,461,76
4,광진구,2588,35,57,100,187,98,52,675,465,712,175,32


In [ ]:
CCTV_Seoul['총계'] = CCTV_Seoul['총계'].astype(int)
CCTV_Seoul['2012년 이전'] = CCTV_Seoul['2012년 이전'].astype(int)
CCTV_Seoul['2020년'] = CCTV_Seoul['2020년'].astype(int)

CCTV_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구분        25 non-null     object
 1   총계        25 non-null     int32 
 2   2012년 이전  25 non-null     int32 
 3   2012년     25 non-null     int64 
 4   2013년     25 non-null     int64 
 5   2014년     25 non-null     int64 
 6   2015년     25 non-null     int64 
 7   2016년     25 non-null     int64 
 8   2017년     25 non-null     int64 
 9   2018년     25 non-null     int64 
 10  2019년     25 non-null     int64 
 11  2020년     25 non-null     int32 
 12  2021년     25 non-null     int64 
dtypes: int32(3), int64(9), object(1)
memory usage: 2.4+ KB


짜잔! 잘 바뀌었습니다. dtype이 int32로 바뀐 이유는 pandas가 알아서 현재 데이터를 보고 메모리를 적게 소모하는 int 중 int32로 바꿨기 때문이예요. 

(나중에 대용량 데이터를 처리하게 되면 astype에 np.int8, np.int16 등으로 메모리를 줄여서 연산할 수 있어요. 물론 그쯤 되면 numpy Array를 사용하실지도 모르겠네요.)


그런데 구의 column이름이 '기관명'으로 되어 있네요. '구별'로 바꿔보겠습니다.

In [ ]:
CCTV_Seoul.rename(columns={CCTV_Seoul.columns[0] : '구별'}, 
                  inplace=True)
CCTV_Seoul.head()

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
0,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
1,중 구,2333,16,114,87,77,236,240,372,386,155,361,289
2,용산구,2383,34,71,234,125,221,298,351,125,307,617,0
3,성동구,3602,448,125,212,105,339,310,874,390,262,461,76
4,광진구,2588,35,57,100,187,98,52,675,465,712,175,32


서울인구수가 담긴 엑셀 데이터는 read_excel로 가져와 보겠습니다.

In [ ]:
# pip install xlrd

In [ ]:
pop_Seoul = pd.read_excel('/Report.xls')
pop_Seoul.head()  # 데이터 앞의 5줄만 보여줘!

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,인구밀도,인구밀도.1,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,인구밀도,인구밀도,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,인구밀도(명/㎢),면적(㎢),세대당인구,65세이상고령자
2,2012,합계,4177970,10442426,5159665,5282761,10195318,5041336,5153982,247108,118329,128779,17255,605.18,2.44,1110995
3,2012,종로구,75659,173148,85997,87151,165207,82274,82933,7941,3723,4218,7243,23.91,2.18,23868
4,2012,중구,61546,140807,70762,70045,133360,67043,66317,7447,3719,3728,14136,9.96,2.17,18875


혹시 
> ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

이런 에러가 나온다면 코드 실행 전에 

`pip install xlrd` 를 실행시켜보세요.

역시나 column 제목들이 마음에 안들죠? index 1번째 줄에 있는 것으로 타이틀을 변경하고 싶고, 필요한 컬럼만 가져오고 싶어요. 그럴 때는 아래와 같이 하시면 됩니다.

In [ ]:
pop_Seoul = pd.read_excel('/Report.xls', 
                          header= 2,                       # header는 row 2번째줄로!
                          usecols = 'A, B, D, E, F, G, J, M, N, O, P',       # 요 columns만 가져온다!
                          )  
pop_Seoul.head()

,기간,자치구,계,남자,여자,계.1,계.2,인구밀도(명/㎢),면적(㎢),세대당인구,65세이상고령자
0,2012,합계,10442426,5159665,5282761,10195318,247108,17255,605.18,2.44,1110995
1,2012,종로구,173148,85997,87151,165207,7941,7243,23.91,2.18,23868
2,2012,중구,140807,70762,70045,133360,7447,14136,9.96,2.17,18875
3,2012,용산구,255294,124708,130586,243232,12062,11674,21.87,2.20,33027
4,2012,성동구,306868,153272,153596,299604,7264,18214,16.85,2.38,34180


 그런데 columns 이름이 마음에 들지 않아요. 그리고 이 표를 잘 보세요. 우리가 나중에 구별 인구 대비 CCTV 설치수까지 비교를 해본다고 했잖아요. 나중에 이 데이터를 합칠텐데 CCTV 자료는 CCTV_Seoul에 있고, 서울인구 자료는 pop_Seoul에 나뉘어있죠? 데이터를 합치는 기준을 지역구로 하고 싶어요. 그래서 자치구를 '구별'로 바꿔주고 컬럼을 변수의 특성이 더 잘 나타나게 바꾸려고 합니다.

In [ ]:
pop_Seoul = pop_Seoul.rename(columns={pop_Seoul.columns[1] : '구별',
                                      pop_Seoul.columns[2] : '인구수',
                                      pop_Seoul.columns[3] : '남성',
                                      pop_Seoul.columns[4] : '여성',
                                      pop_Seoul.columns[5] : '한국인',
                                      pop_Seoul.columns[6] : '외국인',
                                      pop_Seoul.columns[7] : '인구밀도',
                                      pop_Seoul.columns[8] : '면적',
                                      #pop_Seoul.columns[9] : '세대당인구',
                                      pop_Seoul.columns[10] : '고령자'})
pop_Seoul.head()

,기간,구별,인구수,남성,여성,한국인,외국인,인구밀도,면적,세대당인구,고령자
0,2012,합계,10442426,5159665,5282761,10195318,247108,17255,605.18,2.44,1110995
1,2012,종로구,173148,85997,87151,165207,7941,7243,23.91,2.18,23868
2,2012,중구,140807,70762,70045,133360,7447,14136,9.96,2.17,18875
3,2012,용산구,255294,124708,130586,243232,12062,11674,21.87,2.20,33027
4,2012,성동구,306868,153272,153596,299604,7264,18214,16.85,2.38,34180


pop_Seoul도 info를 통해 이것저것 확인해줍시다.

In [ ]:
pop_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기간      260 non-null    int64  
 1   구별      260 non-null    object 
 2   인구수     260 non-null    int64  
 3   남성      260 non-null    int64  
 4   여성      260 non-null    int64  
 5   한국인     260 non-null    int64  
 6   외국인     260 non-null    int64  
 7   인구밀도    260 non-null    object 
 8   면적      260 non-null    object 
 9   세대당인구   260 non-null    float64
 10  고령자     260 non-null    int64  
dtypes: float64(1), int64(7), object(3)
memory usage: 22.5+ KB


null 값은 없고, 인구밀도, 면적이 object네요. 바로 바꿔줍시다.

In [ ]:
pop_Seoul['인구밀도'] = pop_Seoul['인구밀도'].astype(int)
pop_Seoul['면적'] = pop_Seoul['면적'].astype(float)    # 면적은 제곱 km, 소수점 2째 자리까지 있어서 float

pop_Seoul.info()

ValueError: invalid literal for int() with base 10: '-'

헉! 2015년 이전까지만 인구밀도, 면적이 제공되네요. 아무래도 이번 분석의 목적은 10년 동안의 CCTV 개수 변화에 어떤 요인이 영향을 끼쳤는지니까 사용하지 못할 것 같습니다. 인구밀도, 면적 컬럼을 삭제해 줍시다😥

In [ ]:
pop_Seoul = pop_Seoul.drop('인구밀도', axis=1)
pop_Seoul = pop_Seoul.drop('면적', axis=1)

pop_Seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기간      260 non-null    int64  
 1   구별      260 non-null    object 
 2   인구수     260 non-null    int64  
 3   남성      260 non-null    int64  
 4   여성      260 non-null    int64  
 5   한국인     260 non-null    int64  
 6   외국인     260 non-null    int64  
 7   세대당인구   260 non-null    float64
 8   고령자     260 non-null    int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 18.4+ KB


### 4.2. 탐색적 데이터 분석Exploratory Data Analysis

**탐색적 데이터분석**, 줄여서 **EDA**는 데이터 분석에 돌입하기 전 데이터를 이해하기 위한 모든 작업입니다. 데이터의 형태나 분포가 어떤지, 어떤 양상을 띄는지 등 기본적인 표, 그래프를 그리는 일이 해당합니다. 누군가는 코드 치기도 바쁘고, 마감도 얼마 안남았는데 굳이 해야하나 싶기도 하겠지만! 꼭 필요한 작업이예요. 잘 했다고 생각한 분석이 알고보니 전혀 그런 데이터가 아닐 수도 있거든요. 심하게는 "이 사람은 암에 걸리지 않았을겁니다!"라고 자신만만하게 분석했는데, 암 관련 데이터가 아니거나 암 치료와 관련된 데이터라면 지금까지 한 모든 것들이 물거품이 되어버리거든요. 뿐만 아니라 EDA를 통해 데이터에 대해 더 자세하게 이해할 수 있으니 분석에 대한 통찰을 얻을 수 있겠죠?

#### 데이터 확인하기

모든 구가 다 있는지 확인하고 넘어갑시다.

- .unique(): 한번이라도 나타난 데이터를 보여줍니다.

In [ ]:
CCTV_Seoul['구별'].unique()

array(['종로구', '중 구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

CCTV 데이터의 중구는 '중 구'로 되어있군요. 전에 배운 replace를 통해 '중 구'를 '중구'로 바꿔봅시다.

In [ ]:
CCTV_Seoul.replace('중 구', '중구', inplace=True)

CCTV_Seoul['구별'].unique()

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [ ]:
pop_Seoul['구별'].unique()

array(['합계', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구',
       '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구',
       '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'],
      dtype=object)

In [ ]:
pop_gu = pop_Seoul['구별'].unique()
pop_gu[1:]

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [ ]:
print(len(pop_gu[1:]))
print(len(CCTV_Seoul['구별'].unique()))

25
25


CCTV와 POP 모두 25개 구가 잘 들어 있군요. 혹시 둘이 다를 수 있으니 == 을 이용해 같은지 알아봅시다.

In [ ]:
CCTV_Seoul['구별'].unique() == pop_gu[1:]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

일치하네요. 좋아요!

#### 결측치 처리

아까 info()를 통해 nan값이 있는지 확인 했지만 isnull(), isna() 등 다른 방법으로도 nan값을 확인할 수 있답니다.

- isnull()
- isna()

In [ ]:
pop_Seoul[pop_Seoul['구별'].isnull()] 

,기간,구별,인구수,남성,여성,한국인,외국인,세대당인구,고령자


아까 확인한 것처럼 null값은 없네요! 혹시 null값이 있다면 이때 제거해주시면 됩니다.

또, 앞으로 '합계'는 필요할 것 같지 않아서 해당 row는 지워줄께요.

In [ ]:
# 삭제할 row의 index 찾기
drop_index = pop_Seoul[pop_Seoul['구별'] == '합계'].index

# drop
pop_Seoul = pop_Seoul.drop(drop_index)

In [ ]:
# '구별'이 '합계'인 개수가 몇이니? -> 0이요! -> 잘 삭제했구나!
np.sum(pop_Seoul['구별'] == '합계')

0

#### 데이터 정렬

CCTV가 어디에 많이 있고 없는지 궁금하죠? 그러면 '총계'가 CCTV 수니까 이를 정렬시켜보겠습니다.

- sort_values(by='기준 칼럼', ascending=True): 기준 칼럼을 기준으로 오름차순 정렬하기

In [ ]:
CCTV_Seoul.sort_values('총계', ascending=True).head()

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
9,도봉구,1629,39,22,96,181,79,159,134,222,198,168,331
0,종로구,1772,813,0,0,210,150,1,261,85,9,200,43
19,동작구,2297,41,24,25,503,128,253,271,300,322,419,11
1,중구,2333,16,114,87,77,236,240,372,386,155,361,289
17,금천구,2374,0,0,178,80,361,133,196,540,369,508,9


데이터를 살펴보는 것이니 따로 변수에 담아줄 필요는 없어요. 보니까 도봉구, 종로구, 동작구, 중구, 금천구가 CCTV 수로 하위 TOP5네요. 그리고 옆에 인덱스를 보면 구별에 따라서 인덱스가 적혀져 있어요. 이렇게 판다스는 고유한 인덱스가 변하지 않고 따라온다는 사실 덕분에 편하게 데이터를 가공할 수 있어요.

도봉구, 금천구는 각각 북쪽, 서쪽 외곽이죠. 종로구와 중구는 강북의 중심지역이네요. 그리고 우리 학교가 있는 동작구가 CCTV가 적은 축에 속하는군요.

정확히는 모르겠지만 외곽이나 큰 상권이 없거나 유동인구가 비교적 적은 곳이 많은 것 같습니다. 뭐, 더 자세한 내용은 분석을 해봐야 알겠지만요. 이런 생각을 할 수 있게 해준다는 점에서 EDA가 중요하답니다.

반대로 CCTV가 많이 설치되어 있는 TOP5 구들을 봅시다.

In [ ]:
CCTV_Seoul.sort_values(by='총계', ascending=False).head(5) 

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년
22,강남구,6502,124,77,75,597,840,1310,999,748,789,942,1
20,관악구,4942,428,205,291,513,529,621,687,663,640,331,34
16,구로구,4075,852,216,349,187,268,326,540,488,434,415,0
7,성북구,3958,83,78,170,230,323,594,460,867,714,251,188
11,은평구,3791,14,3,44,332,329,555,403,635,1057,288,131


강남과 구로는 이전과 다르게 21년에는 거의 설치하지 않았군요. 코로나19 방역 때문에 인력과 예산이 부족하거나, 유동인구가 줄어서라고 생각할 수도 있겠습니다.

강남구, 관악구, 구로구, 성북구, 은평구 순이군요. 강남은 돈과 사람이 많은 곳이라서 맞는 것 같은데, 나머지 곳들은 딱히 해당하지는 않네요. 관악구, 구로구, 은평구도 외곽에 해당하니 외곽지역 가설도 생각을 조금 더 해봐야겠어요. 이렇게 EDA는 가설을 설정하고 수정하는데 도움을 준답니다.

#### 데이터 연산하기

이 시점에서 저는 

- 최근 5년 사이에 CCTV가 얼마나 증가했는지
- 인구, 남성, 여성, 한국인, 외국인, 인구밀도, 면적, 세대당인구, 고령자가 많을 수록 CCTV가 많은지(인구가 CCTV 설치에 영향을 주는지)

정도가 궁금하네요. 물론 평균 소득이나 세수를 같이 보면 좋겠지만 개인정보 때문에 제공하지 않거나 전처리가 복잡해요. 아직 할 일이 많기 때문에 추후 자율 과제로 남기겠습니다.

그럼 5년간 CCTV 증가율을 구해봅시다.

$$최근5년간 CCTV 증가율 = \frac{\sum_{i=2017}^{2021} CCTV_i} {\sum_{i=2012이전}^{2016} CCTV_i} \times 100$$ 

- **(최근 5년간 CCTV 증가율) = (2017년 + 2018년 + 2019년 + 2020년 + 2021년) / (2016년도 이전) * 100**

In [ ]:
# 최근 5년간 CCTV 설치 증가율
CCTV_Seoul['최근증가율'] = ((CCTV_Seoul['2017년'] + CCTV_Seoul['2018년'] 
                       + CCTV_Seoul['2019년'] + CCTV_Seoul['2020년'] + CCTV_Seoul['2021년'])
                       / (CCTV_Seoul['2012년 이전'] + CCTV_Seoul['2013년'] + CCTV_Seoul['2014년']
                        + CCTV_Seoul['2015년'] + CCTV_Seoul['2016년'])
                       ) * 100
CCTV_Seoul.sort_values(by='최근증가율', ascending=False).head(5)

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,2019년,2020년,2021년,최근증가율
8,강북구,2462,0,0,24,65,105,243,6,392,1000,588,39,463.386728
4,광진구,2588,35,57,100,187,98,52,675,465,712,175,32,436.228814
23,송파구,2854,72,61,86,85,215,148,241,542,1068,235,101,360.891089
6,중랑구,3296,302,24,253,88,141,161,162,173,1049,939,4,246.243386
1,중구,2333,16,114,87,77,236,240,372,386,155,361,289,238.262195


#### 데이터 탐색

전체 인구 중 여성, 외국인, 고령자 비율을 계산해 봅시다. 인구 특성을 반영했는지 알아보는거예요!

In [ ]:
pop_Seoul['여성비율'] = pop_Seoul['여성'] / pop_Seoul['인구수'] * 100 #여성비율 칼럼 생성
pop_Seoul['외국인비율'] = pop_Seoul['외국인'] / pop_Seoul['인구수'] * 100 #외국인비율 칼럼 생성
pop_Seoul['고령자비율'] = pop_Seoul['고령자'] / pop_Seoul['인구수'] * 100 #고령자비율 칼럼 생성
pop_Seoul.head()

,기간,구별,인구수,남성,여성,한국인,외국인,세대당인구,고령자,여성비율,외국인비율,고령자비율
1,2012,종로구,173148,85997,87151,165207,7941,2.18,23868,50.333241,4.586250,13.784739
2,2012,중구,140807,70762,70045,133360,7447,2.17,18875,49.745396,5.288800,13.404873
3,2012,용산구,255294,124708,130586,243232,12062,2.20,33027,51.151222,4.724749,12.936849
4,2012,성동구,306868,153272,153596,299604,7264,2.38,34180,50.052791,2.367142,11.138340
5,2012,광진구,384269,189367,194902,371313,12956,2.34,35759,50.720199,3.371596,9.305721


오름차순, 내림차순으로 알아봅시다.

In [ ]:
pop_Seoul.sort_values(by='인구수', ascending=False).head(5)

,기간,구별,인구수,남성,여성,한국인,외국인,세대당인구,고령자,여성비율,외국인비율,고령자비율
206,2019,송파구,682741,330412,352329,675961,6780,2.43,87334,51.605074,0.993056,12.791674
24,2012,송파구,680150,333060,347090,673115,7035,2.61,58490,51.031390,1.034331,8.599574
50,2013,송파구,674955,330030,344925,668415,6540,2.60,61745,51.103407,0.968953,9.148017
232,2020,송파구,673926,325080,348846,667960,5966,2.37,93483,51.763250,0.885260,13.871404
180,2018,송파구,673507,326849,346658,666635,6872,2.46,81364,51.470586,1.020331,12.080647


이런! 여러 시간대의 데이터가 같이 있어서 인구가 제일 많은 송파구만 나오는군요. 우리는 시기별로 인구가 많은 곳을 순서대로 알고 싶잖아요. 이럴때는 시각화하면 한눈에 알아볼 수 있답니다. 그 전에 저는 기간, 구별로 groupby를 해 줄거예요.

In [ ]:
import datetime

pop_Seoul['기간'] = pop_Seoul['기간'].astype(str)
pop_Seoul['기간'] = pd.to_datetime(pop_Seoul['기간'])

In [ ]:
pop_Seoul['기간'] = pop_Seoul['기간'].dt.year

In [ ]:
pop_Seoul_gby = pop_Seoul.groupby(['기간','구별']).sum()
pop_Seoul_gby# = pd.to_DataFrame(pop_Seoul_gby)

인구수      남성      여성     한국인    외국인  세대당인구    고령자       여성비율  \
기간   구별                                                                    
2012 강남구  569997  274124  295873  564197   5800   2.45  50348  51.907817   
     강동구  492728  246237  246491  487905   4823   2.60  44941  50.025775   
     강북구  346493  171099  175394  343157   3336   2.41  46222  50.619782   
     강서구  573794  282185  291609  567431   6363   2.54  55821  50.821201   
     관악구  540520  273640  266880  523029  17491   2.11  56620  49.374676   
...          ...     ...     ...     ...    ...    ...    ...        ...   
2021 용산구  237285  115085  122200  222953  14332   2.01  39070  51.499252   
     은평구  477173  228024  249149  473307   3866   2.20  87241  52.213558   
     종로구  153789   74186   79603  144683   9106   1.97  27818  51.761179   
     중구   131787   64083   67704  122499   9288   1.93  24392  51.373808   
     중랑구  391885  192903  198982  387350   4535   2.09  71682  50.775610   

             외국인비율      고령자비율  
기간   구별                        
2012 강남구  1.017549   8.833029  
     강동구  0.978836   9.120854  
     강북구  0.962790  13.339952  
     강서구  1.108935   9.728404  
     관악구  3.235958  10.475098  
...            ...        ...  
2021 용산구  6.039994  16.465432  
     은평구  0.810188  18.282887  
     종로구  5.921100  18.088420  
     중구   7.047736  18.508654  
     중랑구  1.157227  18.291591  

[250 rows x 10 columns]

#### 데이터 병합하기

이렇게 데이터를 하나씩 보면서 인구 데이터와 CCTV 데이터의 특징을 살펴봤어요. 그런데 아직 우리는 인구 대비 CCTV 현황을 보지 못했죠? 이 데이터는 따로 존재하기 때문에 두 데이터를 병합해야 해요. 판다스에서 merge()를 배웠으니, 이것을 사용하여 데이터를 옆으로 병합해 보겠습니다.

In [ ]:
# data_result라는 새로운 변수에 담아줄게요! 
data_result = pd.merge(CCTV_Seoul, pop_Seoul, on='구별')
data_result.head()

,구별,총계,2012년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,...,인구수,남성,여성,한국인,외국인,세대당인구,고령자,여성비율,외국인비율,고령자비율
0,종로구,1772,813,0,0,210,150,1,261,85,...,173148,85997,87151,165207,7941,2.18,23868,50.333241,4.586250,13.784739
1,종로구,1772,813,0,0,210,150,1,261,85,...,167867,83211,84656,160070,7797,2.17,23997,50.430400,4.644749,14.295246
2,종로구,1772,813,0,0,210,150,1,261,85,...,165344,81439,83905,156993,8351,2.15,24537,50.745718,5.050682,14.839970
3,종로구,1772,813,0,0,210,150,1,261,85,...,163822,80531,83291,154986,8836,2.13,24892,50.842378,5.393659,15.194540
4,종로구,1772,813,0,0,210,150,1,261,85,...,161922,79322,82600,152737,9185,2.12,25091,51.012216,5.672484,15.495733


이제 앞에서 column이름은 구별로 공통적으로 처리했는지 아시겠죠? 두 데이터의 공통 컬럼인 '구별'로 merge를 하면 공통된 key를 가진 데이터만 남게 됩니다.

#### 데이터 삭제하기

그러면 이제 분석에 필요없는 칼럼들을 지워봅시다. 다른 지표가 현재 개수와 최근 증가율에 영향을 끼쳤다는 가설을 기반으로 분석할 거예요. '소계','최근증가율' 빼고는 필요하지 않으니까 drop()을 사용하여 지워보겠습니다. drop()은 기본이 axis=0이니까 column을 삭제하려면 axis=1로 해야겠죠?

In [ ]:
data_result = data_result.drop(['2012년 이전', '2012년', '2013년', '2014년',
                                '2015년', '2016년', '2017년', '2018년', '2019년',
                                '2020년', '2021년', 
                                '남성', '여성', '여성비율'], axis = 1)


In [ ]:
data_result.head()

,구별,총계,최근증가율,기간,인구수,한국인,외국인,세대당인구,고령자,외국인비율,고령자비율
0,종로구,1772,50.936968,2012,173148,165207,7941,2.18,23868,4.586250,13.784739
1,종로구,1772,50.936968,2013,167867,160070,7797,2.17,23997,4.644749,14.295246
2,종로구,1772,50.936968,2014,165344,156993,8351,2.15,24537,5.050682,14.839970
3,종로구,1772,50.936968,2015,163822,154986,8836,2.13,24892,5.393659,15.194540
4,종로구,1772,50.936968,2016,161922,152737,9185,2.12,25091,5.672484,15.495733


엥? 종로구가 여러개 들어가 있죠? 하지만 총계, 최근 증가율을 제외한 나머지 값이 조금씩 달라요. 연도별 데이터가 다 들어가 있어서 그래요. 일단 구별, 총계, 최근증가율, 기간별로 groupby 해보겠습니다.

In [ ]:
grouped = data_result.groupby(['구별', '기간'])
data_result_gby = grouped['인구수','세대당인구','외국인비율','고령자비율', '총계', '최근증가율'].sum()
data_result_gby

<ipython-input-180-a40c0bf839b8>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_result_gby = grouped['인구수','세대당인구','외국인비율','고령자비율', '총계', '최근증가율'].sum()


인구수  세대당인구     외국인비율      고령자비율    총계       최근증가율
구별  기간                                                        
강남구 2012  569997   2.45  1.017549   8.833029  6502  118.092329
    2013  569152   2.44  0.975662   9.393097  6502  118.092329
    2014  583446   2.44  0.913881   9.845641  6502  118.092329
    2015  581760   2.43  0.905012  10.370256  6502  118.092329
    2016  572140   2.42  0.878282  10.824449  6502  118.092329
...          ...    ...       ...        ...   ...         ...
중랑구 2017  412780   2.28  1.103251  14.356800  3296  246.243386
    2018  408147   2.23  1.209858  15.148954  3296  246.243386
    2019  402024   2.19  1.245946  16.208983  3296  246.243386
    2020  399562   2.12  1.216332  17.380782  3296  246.243386
    2021  391885   2.09  1.157227  18.291591  3296  246.243386

[250 rows x 6 columns]

그리고 비율이나 개수 변화를 중심으로 보기 위해 21년 기준으로 data_result_gby21도 만들어줄께요.

In [ ]:
data_result21 = data_result[data_result['기간']==2021]
data_result21.head()

,구별,총계,최근증가율,기간,인구수,한국인,외국인,세대당인구,고령자,외국인비율,고령자비율
9,종로구,1772,50.936968,2021,153789,144683,9106,1.97,27818,5.921100,18.088420
19,중구,2333,238.262195,2021,131787,122499,9288,1.93,24392,7.047736,18.508654
29,용산구,2383,153.508772,2021,237285,222953,14332,2.01,39070,6.039994,16.465432
39,성동구,3602,145.898161,2021,292672,285990,6682,2.13,46380,2.283102,15.847092
49,광진구,2588,436.228814,2021,352627,339996,12631,2.02,51723,3.581972,14.667907


In [ ]:
grouped = data_result21.groupby('구별')
data_result_gby21 = grouped['인구수','세대당인구','외국인비율','고령자비율', '총계', '최근증가율'].sum()
data_result_gby21.head()

<ipython-input-194-0f0d066cea02>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_result_gby21 = grouped['인구수','세대당인구','외국인비율','고령자비율', '총계', '최근증가율'].sum()


,인구수,세대당인구,외국인비율,고령자비율,총계,최근증가율
구별,,,,,,
강남구,537800,2.28,0.884716,14.545556,6502,118.092329
강동구,466472,2.29,0.816341,15.878767,2547,180.000000
강북구,302563,2.07,1.117453,21.262679,2462,463.386728
강서구,579768,2.12,0.940549,15.964662,2560,233.013699
관악구,499449,1.76,2.753034,15.991823,4942,98.866499


### 상관계수 분석

이제 간단한 **상관계수 분석**을 해볼거에요. 상관관계는 관계를 알고자 하는 두 변수가 모두 **연속변수**일 경우에만 계산이 가능하고, 이는 인과관계를 분석하는게 아닌 두 연속변수가 관련이 있는지 없는지를 알아보는 **기술통계** 분석이에요. 쉽게 말해서 **상관계수의 절대값이 크다고 원인이라고 단정지을 수 없다**는 거죠.

두 변수를 x,y라고 하면 상관관계 계수(coefficient)는 x값과 y값을 모두 z값으로 표준화해서 Zx * Zy를 모두 더한다음, n(총 개수)으로 나눠준 것이 상관계수에요. 다른 것은 다됐고 여기서 상관계수는 넘파이가 다 계산해 줄 테니까 우리는 여기서 상관계수가 무엇을 의미하는지만 알면 됩니다!

- 0 ~ 0.2 : 매우 약한 상관관계
- 0.2 ~ 0.4 : 약한 상관관계
- 0.4 ~ 0.6 : 어느 정도의 상관관계
- 0.6 ~ 0.8 : 강한 상관관계
- 0.8 ~ 1.0 : 매우 강한 상관관계

상관계수의 범위는 -1부터 1까지 음수가 나오면 음의 상관관계, 양수가 나오면 양의 상관관계라는 것까지만 알면 됩니다.

그럼 다수의 데이터 중 상관계수가 가장 큰 값은 데이터를 비교해볼게요. numpy의 corrcoef 명령을 사용하면 상관계수를 알아서 계산해줍니다.

In [ ]:
# 약한 음의 상관관계 
np.corrcoef(data_result_gby['고령자비율'], data_result_gby['총계'])

array([[ 1.        , -0.26592195],
       [-0.26592195,  1.        ]])

In [ ]:
# 상관관계가 거의 없다고 할 수 있음. 
np.corrcoef(data_result_gby['외국인비율'], data_result_gby['총계'])

array([[ 1.        , -0.10597348],
       [-0.10597348,  1.        ]])

In [ ]:
# 0.46: 어느 정도의 양의 상관관계 
np.corrcoef(data_result_gby['인구수'], data_result_gby['총계'])

array([[1.        , 0.46415004],
       [0.46415004, 1.        ]])